# [初心者のためのTensorFlow2.0入門](https://www.tensorflow.org/tutorials/quickstart/beginner?hl=ja)を実践

参考文献\
[初心者のためのTensorFlow2.0入門](https://www.tensorflow.org/tutorials/keras/classification?hl=ja)\
[TensorFlowのチュートリアルがディープラーニングの勉強のスタートに良かったのでオススメしたい](https://qiita.com/Kuma_T/items/4449f008cad18fbb7f1a)\
[Keras Documentation](https://keras.io/ja/)\
ゼロから作るDeeo Learning

## tensorflowをインポートする

In [1]:
import tensorflow as tf

## MNISTデータセットを読み込む


*   tf.keras.dataset.mnist : kerasが用意してくれているmnistデー
タセットを変数mnistに代入

*   load_data()は、（x_train：訓練画像，y_train：訓練ラベル），（x_test：テスト画像，y_test：テストラベル）という形式で、読み込んだMNISTデータセットの値を返す

*   サンプルは整数だけど、それを255で割って浮動小数点化に変換する→→データの値の範囲が0から1になるように正規化




In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255.0
x_test = x_test / 255.0

11501568/11490434 [==============================] - 0s 0us/step


## Seqentialモデルを構築


*   Sequentialモデル……シンプルな一直線のモデル（１入力１出力）
* tf.keras.layers.Flatten(input_shape = (28,28))……入力層28×28の入力データを784の1次元データ(28✕28=784)に平滑化
*   tf.keras.layers.Dense(128, activation='relu')……中間層。活性化関数にRelu関数を使う。ニューロンの個数は128個
*   tf.keras.layers.Dropout(0.2)……過学習防止のため、ドロップアウト率=0.2でドロップアウト
*   tf.keras.layers.Dense(10)……出力層。ニューロン数は10個で線形関数により出力の値が計算される。







In [3]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape = (28,28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

## 入力画像１枚に対するロジットや対数オッズ比と呼ばれるスコアを算出

In [4]:
predictions = model(x_train[:1]).numpy()
predictions

array([[-0.23820132,  0.47049275, -0.9763349 ,  0.9952246 ,  0.13406426,
        -0.5500001 , -0.34624022,  0.34882078,  0.6692051 , -0.37388715]],
      dtype=float32)

## 上の出力値をクラスごとに確率に変換

In [5]:
tf.nn.softmax(predictions).numpy()

array([[0.06590793, 0.13388121, 0.03150434, 0.22625999, 0.09563362,
        0.0482531 , 0.05915849, 0.11854362, 0.16331245, 0.05754533]],
      dtype=float32)

In [6]:
#print(x_train.shape)
print(x_train[:1].shape)

(1, 28, 28)


In [7]:
print(x_train.shape)

(60000, 28, 28)


## ラベルと予測値での（スパース）クロスエントロピーの計算


In [8]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
loss_fn(y_train[:1], predictions).numpy()

3.0312953

## モデルのコンパイル
モデルのコンパイルとは、自分で決めたモデル（今の場合、1入力1出力で128ユニット＝128個のニューロンで構成された中間層としたSequentialモデル）を「どのように」学習をするかを詳細に設定すること。


*   optimizer……最適化手法のこと。adamはそのうちの1つ
*   loss……損失関数
*   metrics……評価指標。Accuracyはそのうちの1つ





In [9]:
model.compile(optimizer='adam',
              loss = loss_fn,
              metrics = ['accuracy'])

## 学習
学習……損失を最小化するようにモデルのパラメータを調整
1epochは学習においてデータをすべて使いきったときの回数に対応する。つまり、1回の（ミニバッチ）学習に使うデータ数を表す


*   model.fitは、損失を最小化するようにモデルのパラメータを調整するメソッド



In [10]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.2947 - accuracy: 0.9149
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1385 - accuracy: 0.9588
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1038 - accuracy: 0.9680
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0840 - accuracy: 0.9737
Epoch 5/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0734 - accuracy: 0.9773


## モデルの性能を検証


*   model.evaluateはテスト用データを用いてモデルの性能を検査する関数。モデルの損失値と評価値を返す


In [14]:
model.evaluate(x_test, y_test, verbose=1)

313/313 [==============================] - 1s 3ms/step - loss: 0.0672 - accuracy: 0.9807


[0.06724967062473297, 0.9807000160217285]